# PYTORCH CNN Classifier

To run this notebook on an another benchmark, use

```
papermill utils/torch_cnn_classifier.ipynb torch_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [1]:
# DATASET = 'no_dataset'
DATASET = 'demo_human_or_worm'
VERSION = 0
BATCH_SIZE = 64
EPOCHS = 1

In [2]:
# Parameters
DATASET = "human_enhancers_ensembl"


In [3]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

human_enhancers_ensembl 0 64 1


## Config

In [4]:
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer

from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.dataset_getters.pytorch_datasets import get_dataset
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.models.torch import CNN
from genomic_benchmarks.dataset_getters.utils import coll_factory, LetterTokenizer, build_vocab, check_seq_lengths, check_config, VARIABLE_LENGTH_DATASETS

In [5]:
USE_PADDING = DATASET in VARIABLE_LENGTH_DATASETS

## Choose the dataset

In [6]:
if not is_downloaded(DATASET):
    download_dataset(DATASET, local_repo=True)

/home/katarina/git/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


Downloading...
From: https://drive.google.com/uc?id=1gZBEV_RGxJE8EON5OObdrp5Tp8JL0Fxb
To: /home/katarina/.genomic_benchmarks/human_enhancers_ensembl.zip



  0%|                                                                                                                                                       | 0.00/51.1M [00:00<?, ?B/s]


  3%|████▎                                                                                                                                         | 1.57M/51.1M [00:00<00:03, 14.8MB/s]


  9%|█████████████                                                                                                                                 | 4.72M/51.1M [00:00<00:02, 18.1MB/s]

...


 34%|████████████████████████████████████████████████                                                                                              | 17.3M/51.1M [00:00<00:01, 27.4MB/s]


 50%|███████████████████████████████████████████████████████████████████████▎                                                                      | 25.7M/51.1M [00:00<00:00, 30.5MB/s]


 67%|██████████████████████████████████████████████████████████████████████████████████████████████▋                                               | 34.1M/51.1M [00:01<00:00, 35.1MB/s]


 83%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                        | 42.5M/51.1M [00:01<00:00, 39.2MB/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 51.1M/51.1M [00:01<00:00, 38.3MB/s]

In [7]:
info(DATASET, local_repo=True)

Dataset `human_enhancers_ensembl` has 2 classes: negative, positive.

The length of genomic intervals ranges from 2 to 573, with average 268.8641324705183 and median 269.0.

Totally 154842 sequences have been found, 123872 for training and 30970 for testing.


,train,test
negative,61936,15485
positive,61936,15485


In [8]:
train_dset = get_dataset(DATASET, 'train')

In [9]:
NUM_CLASSES = len(set([train_dset[i][1] for i in range(len(train_dset))]))
NUM_CLASSES

2

## Tokenizer and vocab

In [10]:
tokenizer = get_tokenizer(LetterTokenizer())
vocabulary = build_vocab(train_dset, tokenizer, use_padding=USE_PADDING)

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 9
{'<pad>': 8, 'T': 5, '<eos>': 6, 'G': 3, 'A': 4, 'C': 2, '<bos>': 1, 'N': 7, '<unk>': 0}


## Dataloader and batch preparation

In [11]:
# Run on GPU or CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_seq_len, nn_input_len = check_seq_lengths(dataset=train_dset, use_padding=USE_PADDING)

# Data Loader
if(USE_PADDING):
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = nn_input_len)
else:
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = None)

train_loader = DataLoader(train_dset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate)

Using cpu device


max_seq_len  573


not all sequences are of the same length


## Model

In [12]:
model = CNN(
    number_of_classes=NUM_CLASSES,
    vocab_size=vocabulary.__len__(),
    embedding_dim=100,
    input_len=nn_input_len,
    device=device
).to(device)

## Training

In [13]:
model.fit(train_loader, epochs=EPOCHS)

Epoch 0


/home/katarina/git/genomic_benchmarks/src/genomic_benchmarks/dataset_getters/utils.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(pad(x), dtype=torch.long)


Train metrics: 
 Accuracy: 66.1%, Avg loss: 0.648035 



## Testing

In [14]:
test_dset = get_dataset(DATASET, 'test')
test_loader = DataLoader(test_dset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate)

acc, f1 = model.test(test_loader)
acc, f1

Test metrics: 
 Accuracy: 0.663158, F1 score: 0.545486, Avg loss: 0.638594 



(0.6631578947368421, 0.545486232136633)